# Actualizando clientes

Para actualizar un cliente podemos enviar el dni, nombre y apellido a una petición a la ruta `/clientes/modificar/` mediante el método `PUT`.

La diferencia entre `POST` y `PUT` es que las peticiones `PUT` son idempotentes. Es decir, llamar a la misma solicitud `PUT` varias veces siempre producirá el mismo resultado. Mientras que llamar repetidamente a una petición `POST` tiene el efecto secundario de crear el mismo recurso varias veces. Aunque esto en nuestro caso no aplica porque hemos implementado una validación que impide crear dos clientes con el mismo DNI.

En cualquier caso, en lugar del `ModeloCrearCliente` en esta ocasión utilizaremos la versión base `ModeloCliente`, ya que no permitiremos cambiar el DNI y no necesitamos validarlo:

```python
@app.put("/clientes/actualizar/")
async def clientes_actualizar(datos: ModeloCliente):
    if db.Clientes.buscar(datos.dni):
        cliente = db.Clientes.modificar(datos.dni, datos.nombre, datos.apellido)
        if cliente:
            return JSONResponse(content=cliente.to_dict(), headers=headers)
    raise HTTPException(status_code=404)
```

Si hacemos una modificación desde http://127.0.0.1:8000/docs#/default/clientes_actualizar_clientes_actualizar__put:

```json
{
  "dni": "15J",
  "nombre": "Victoria",
  "apellido": "Pérez"
}
```

Debería devolverse el registro modificado:

```json
{"dni":"15J","nombre":"Victoria","apellido":"Pérez"}
```

Y los cambios aparecerán en el fichero:

```csv
15J;Victoria;Pérez
48H;Manolo;López
28Z;Mariana;García
36N;Fernando;López
```

En caso de que el DNI no exista, simplemente se devolverá el error 404 que hemos implementado por defecto:

```json
{
  "detail": "Not Found"
}
```

Y si el nombre o el apellido no cumplen las validaciones mínimas se devolverá el pertinente aviso:

```json
{
  "detail": [
    {
      "loc": [
        "body",
        "apellido"
      ],
      "msg": "ensure this value has at least 2 characters",
      "type": "value_error.any_str.min_length",
      "ctx": {
        "limit_value": 2
      }
    }
  ]
}
```

¡Función de actualizado lista!